In [1]:
from pyspark import SparkConf,SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import Row,SQLContext
import sys
import requests
import nltk
from nltk.classify import NaiveBayesClassifier
from nltk.sentiment import SentimentAnalyzer
from nltk.corpus import subjectivity
from nltk.corpus import stopwords
from nltk.sentiment.util import *
import string
from operator import add
from pyspark.sql.types import StructField, StructType, StringType, IntegerType
import pandas
import socket

In [2]:
# create the Streaming Context from the above spark context with interval size 2 seconds
ssc = StreamingContext(sc, 30)
ip = "localhost"
porta = 9009
resultados = []

In [3]:
# Lendo o arquivo texto e criando um RDD em memória com Spark
arquivo = sc.textFile("/CientistadeDados/Hadoop-Spark/CAP10/2-Arquivos-Cap10.zip/dataset_analise_sentimento/dataset_analise_sentimento.csv")

# Removendo o cabeçalho
header = arquivo.take(1)[0]
dataset = arquivo.filter(lambda line: line != header)

# Essa função separa as colunas em cada linha, cria uma tupla e remove a pontuação.
def get_row(line):
    row = line.split(',')
    sentimento = row[1]
    tweet = row[3].strip()
    translator = str.maketrans({key: None for key in string.punctuation})
    tweet = tweet.translate(translator)
    tweet = tweet.split(' ')
    tweet_lower = []
    for word in tweet:
        tweet_lower.append(word.lower())
    return (tweet_lower, sentimento)

# Aplica a função a cada linha do dataset
dataset_treino = dataset.map(lambda line: get_row(line))

# Cria um objeto SentimentAnalyzer 
sentiment_analyzer = SentimentAnalyzer()

# Certifique-se de ter espaço em disco - Aproximadamente 5GB
# https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml
# nltk.download()
nltk.download("stopwords")

from IPython.display import Image
Image(url = 'ntlkdata.png')

# Obtém a lista de stopwords em Inglês 
stopwords_all = []
for word in stopwords.words('portuguese'):
    stopwords_all.append(word)
    stopwords_all.append(word + '_NEG')

# Obtém 10.000 tweets do dataset de treino e retorna todas as palavras que não são stopwords
dataset_treino_amostra = dataset_treino.take(10000)

all_words_neg = sentiment_analyzer.all_words([mark_negation(doc) for doc in dataset_treino_amostra])
all_words_neg_nostops = [x for x in all_words_neg if x not in stopwords_all]

# Cria um unigram (n-grama) e extrai as features
unigram_feats = sentiment_analyzer.unigram_word_feats(all_words_neg_nostops, top_n = 200)
sentiment_analyzer.add_feat_extractor(extract_unigram_feats, unigrams = unigram_feats)
training_set = sentiment_analyzer.apply_features(dataset_treino_amostra)

# Treinar o modelo
trainer = NaiveBayesClassifier.train
classifier = sentiment_analyzer.train(trainer, training_set)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Bismarck\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Training classifier


In [4]:
resultados = []
palavra = ""

In [5]:
# Essa função classifica os tweets, aplicando as features do modelo criado anteriormente
def classifica_tweet(tweet):
    x = ""
    sentence = [(tweet, '')]
    
    for i in sentence[0][0]:
        x = x + i
    if x == "encerrar":
        return("ENCERRAR", 999)
    
    test_set = sentiment_analyzer.apply_features(sentence)
    return(tweet, classifier.classify(test_set[0][0]))

In [6]:
# Essa função retorna o texto do Twitter
tweet_lower = []
def get_tweet_text(rdd):
    for line in rdd:
        tweet = line.strip()
        translator = str.maketrans({key: None for key in string.punctuation})
        tweet = tweet.translate(translator)
        tweet = tweet.split(' ')
        global tweet_lower
        for word in tweet:
            tweet_lower.append(word.lower())
    return(classifica_tweet(tweet_lower))

In [7]:
def output_rdd(rdd):
    global resultados
    global palavra
    pairs = rdd.map(lambda x: (get_tweet_text(x)[1],1))
    
    counts = pairs.reduceByKey(add)
    output = []
    
    for count in counts.collect():
        output.append(count)
    result = [time.strftime("%I:%M:%S"), output]
    resultados.append(result)

    if len(output) > 0:
        print(result)
        if result[1][0][0] == 999:
#            ssc.stop()
            print("Fim da Execução")
#            sys.exit(1)
            return (True) 


In [8]:
dataStream = ssc.socketTextStream(ip, porta)
#dataStream.pprint()
dataStream.foreachRDD(output_rdd)
# start the streaming computation
ssc.start()
ssc.awaitTermination()

['09:51:01', [(999, 1)]]
Fim da Execução


Py4JJavaError: An error occurred while calling o33.awaitTermination.
: org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "D:\spark-2.4.2\python\pyspark\streaming\util.py", line 74, in call
    if is_instance_of(self.ctx._gateway, r._jrdd, "org.apache.spark.api.java.JavaRDD"):
AttributeError: 'bool' object has no attribute '_jrdd'

	at org.apache.spark.streaming.api.python.TransformFunction.callPythonTransformFunction(PythonDStream.scala:95)
	at org.apache.spark.streaming.api.python.TransformFunction.apply(PythonDStream.scala:78)
	at org.apache.spark.streaming.api.python.PythonDStream$.$anonfun$callForeachRDD$1(PythonDStream.scala:179)
	at org.apache.spark.streaming.api.python.PythonDStream$.$anonfun$callForeachRDD$1$adapted(PythonDStream.scala:179)
	at org.apache.spark.streaming.dstream.ForEachDStream.$anonfun$generateJob$2(ForEachDStream.scala:51)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23

In [ ]:
a  = ['09:21:02', [(999, 1)]]
x = ""
print(a[1][0][0])